#### _Matin's Lab Rotation 3 - BFCO_ | [NEXT](03_community_dFBA_growth_objective.ipynb) | [PREVIOUS](01-1_simulation_isobutyric_strain.ipynb)
# 04. Simulation of the isobutyric Acid Producing Strain

In [1]:
from cameo import load_model
from cameo import fba
from escher import Builder
import pandas as pd
import cobra

/home/matinnu/anaconda3/envs/lab_rotation_3/lib/python3.7/site-packages/sympy/__init__.py:676: SymPyDeprecationWarning: 

importing sympy.core.singleton with 'from sympy import *' has been
deprecated since SymPy 1.6. Use import sympy.core.singleton instead.
See https://github.com/sympy/sympy/issues/18245 for more info.

  deprecated_since_version="1.6").warn()


In [6]:
def simulate_model(experiment_id, model, objective, target, metabolite):
    model.objective = objective
    solution = fba(model)
    f_data = solution.fluxes
    f_data = f_data.rename(experiment_id)
    solution_result = []
    for i in target:
        flux = solution.fluxes.get(i)
        solution_result.append(flux)
    data = [experiment_id] + solution_result
    #print(data)
    df_target = pd.DataFrame(columns=[["Model"]+target], data=[data])
    print("experiment:", experiment_id, "| evaluate compound:", model.metabolites.get_by_id(metabolite).summary())
    return df_target, f_data

def run_series_simulation(experiment, model, mutant, target, metabolite):
    solution_fluxes = pd.DataFrame()
    df = pd.DataFrame()
    for i in experiment.index:
        experiment_id = experiment.loc[i, "experiment_id"]
        if experiment.loc[i, "model_id"] != "model":
            m = mutant
        else:
            m = model
        objective = experiment.loc[i, "objective"]
        data, sol = simulate_model(experiment_id, m, objective, target, metabolite)
        solution_fluxes = solution_fluxes.append(sol)
        df = df.append(data)
        print("****************************************", "\n")
    return df, solution_fluxes

In [2]:
model = load_model('iML1515')
model.solver = 'gurobi'
model

Using license file /home/matinnu/gurobi.lic
Academic license - for non-commercial use only


Name,iML1515
Memory address,0x07f3dd9759c90
Number of metabolites,1877
Number of reactions,2712
Number of groups,0
Objective expression,0.0 + 1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [3]:
mutant = load_model('result/iML1515_isobutyric_strain.xml')
mutant.solver = 'gurobi'
mutant

Name,iML1515
Memory address,0x07f3ddc2ff9d0
Number of metabolites,1880
Number of reactions,2716
Number of groups,0
Objective expression,0.0 + 1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [12]:
mutant.metabolites.get_by_id("3mob_c")

Metabolite identifier,3mob_c
Name,3-Methyl-2-oxobutanoate
Memory address,0x07f3dd8941450
Formula,C5H7O3
Compartment,c
In 6 reaction(s),"VALTA, DHAD1, kivD_heterologous, MOHMT, IPPS, VPAMTr"


In [14]:
experiment = pd.read_csv('data/experiment_isobutyric.txt')
experiment

,experiment_id,model_id,objective
0,wt_glu_xy_ac_biomass,model,BIOMASS_Ec_iML1515_core_75p37M
1,wt_glu_xy_ac_iso,model,DHAD1
2,mt_glu_xy_ac_biomass,mutant,BIOMASS_Ec_iML1515_core_75p37M
3,mt_glu_xy_ac_iso,mutant,DHAD1


In [15]:
target = ["BIOMASS_Ec_iML1515_core_75p37M", "ATPM", "ACODA", "CYSS", "UHGADA", "ACt4pp", "ACKr", "EX_ac_e", "XYLR"]
metabolite = "3mob_c"

df, solution_fluxes = run_series_simulation(experiment, model, mutant, target, metabolite)

experiment: wt_glu_xy_ac_biomass | evaluate compound: 3mob_c
Formula: C5H7O3

Producing Reactions
-------------------
Percent   Flux Reaction                   Definition
100.00% 0.7668    DHAD1  23dhmb_c --> 3mob_c + h2o_c

Consuming Reactions
-------------------
Percent       Flux Reaction                                           Definition
 51.53%    -0.3951     IPPS  3mob_c + accoa_c + h2o_c --> 3c3hmp_c + coa_c + h_c
  0.08% -0.0005911    MOHMT          3mob_c + h2o_c + mlthf_c --> 2dhp_c + thf_c
 48.40%    -0.3711    VALTA               akg_c + val__L_c <=> 3mob_c + glu__L_c
**************************************** 

experiment: wt_glu_xy_ac_iso | evaluate compound: 3mob_c
Formula: C5H7O3

Producing Reactions
-------------------
Percent Flux Reaction                   Definition
100.00% 9.93    DHAD1  23dhmb_c --> 3mob_c + h2o_c

Consuming Reactions
-------------------
Percent  Flux Reaction                              Definition
100.00% -9.93    VALTA  akg_c + val__L_c <=> 3mo

In [19]:
builder = Builder(
    map_json='map/iJO1366.Central metabolism_isobutyric_2.json',
    reaction_data=solution_fluxes.iloc[3],
    #reaction_data=solution.fluxes,
)
builder

Builder(reaction_data={'12DGR120tipp': 0.0, '12DGR140tipp': 0.0, '12DGR141tipp': 0.0, '12DGR160tipp': 0.0, '12…

In [17]:
df

,Model,BIOMASS_Ec_iML1515_core_75p37M,ATPM,ACODA,CYSS,UHGADA,ACt4pp,ACKr,EX_ac_e,XYLR
0,wt_glu_xy_ac_biomass,0.876997,6.86,0.259409,0.217041,0.034126,0.004565,0.510576,0.0,None
0,wt_glu_xy_ac_iso,0.000000,6.86,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,None
0,mt_glu_xy_ac_biomass,0.876908,6.86,0.259382,0.217019,0.034122,0.000000,0.510523,0.0,None
0,mt_glu_xy_ac_iso,0.000000,6.86,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,None


In [18]:
mutant.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,10,6,100.00%
nh4_e,EX_nh4_e,8.386,0,0.00%
o2_e,EX_o2_e,1.614,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-11.61,1,19.36%
h2o_e,EX_h2o_e,-20,0,0.00%
h_e,EX_h_e,-10,0,0.00%
isobuta_e,EX_isobuta_e,-1.614,4,10.76%
val__L_e,EX_val__L_e,-8.386,5,69.88%


In [27]:
media_components = [["EX_xyl__D_e", 5.0],
                    ["EX_glc__D_e", 5.0],
                    #["EX_o2_e", 0.0]
                   ]

medium = model.medium
for i in media_components:
    medium[i[0]] = i[1]
    model.medium = medium
#print(model.medium)

medium = mutant.medium
for i in media_components:
    medium[i[0]] = i[1]
    mutant.medium = medium
#print(mutant.medium)

#mutant.reactions.XYLK.knock_out()

target = ["BIOMASS_Ec_iML1515_core_75p37M", "ATPM", "ACODA", "CYSS", "UHGADA", "ACt4pp", "ACKr", "EX_ac_e", "XYLR"]
metabolite = "3mob_c"
experiment2 = pd.read_csv('data/experiment_isobutyric.txt')
df2, solution_fluxes2 = run_series_simulation(experiment2, model, mutant, target, metabolite)

experiment: wt_glu_xy_ac_biomass | evaluate compound: 3mob_c
Formula: C5H7O3

Producing Reactions
-------------------
Percent   Flux Reaction                   Definition
100.00% 0.1124    DHAD1  23dhmb_c --> 3mob_c + h2o_c

Consuming Reactions
-------------------
Percent       Flux Reaction                                           Definition
 51.53%   -0.05793     IPPS  3mob_c + accoa_c + h2o_c --> 3c3hmp_c + coa_c + h_c
  0.08% -8.667E-05    MOHMT          3mob_c + h2o_c + mlthf_c --> 2dhp_c + thf_c
 48.40%   -0.05441    VALTA               akg_c + val__L_c <=> 3mob_c + glu__L_c
**************************************** 

experiment: wt_glu_xy_ac_iso | evaluate compound: 3mob_c
Formula: C5H7O3

Producing Reactions
-------------------
Percent  Flux Reaction                   Definition
100.00% 7.403    DHAD1  23dhmb_c --> 3mob_c + h2o_c

Consuming Reactions
-------------------
Percent   Flux Reaction                              Definition
100.00% -7.403    VALTA  akg_c + val__L_c <=>

In [28]:
builder = Builder(
    map_json='map/iJO1366.Central metabolism_rotation3.json',
    reaction_data=solution_fluxes2.iloc[2],
    #reaction_data=solution.fluxes,
)
builder

Builder(reaction_data={'12DGR120tipp': 0.0, '12DGR140tipp': 0.0, '12DGR141tipp': 0.0, '12DGR160tipp': 0.0, '12…

In [29]:
df2

,Model,BIOMASS_Ec_iML1515_core_75p37M,ATPM,ACODA,CYSS,UHGADA,ACt4pp,ACKr,EX_ac_e,XYLR
0,wt_glu_xy_ac_biomass,0.128591,6.86,0.038036,0.031824,0.005004,0.000669,-8.137504,8.212368,None
0,wt_glu_xy_ac_iso,0.000000,6.86,0.000000,0.000000,0.000000,0.000000,-1.763333,1.763333,None
0,mt_glu_xy_ac_biomass,0.027678,6.86,0.008187,0.006850,0.001077,0.000000,0.016114,0.000000,None
0,mt_glu_xy_ac_iso,0.000000,6.86,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,None


In [39]:
reaction_1 = mutant.reactions.get_by_id('BIOMASS_Ec_iML1515_core_75p37M')
#reaction_1 = mutant.reactions.get_by_id('ATPM')
reaction_2 = mutant.reactions.get_by_id('EX_isobuta_e')
#reaction_3 = mutant.reactions.get_by_id('EX_xylt_e')
new_objective = {reaction_1: 0.9, reaction_2:0.1}
mutant.objective = new_objective
cobra.manipulation.delete_model_genes(mutant, ['b0733'])
#optimize the model with the new_objective
solution3 = mutant.optimize()
#print_solution(model)

In [40]:
builder = Builder(
    map_json='map/iJO1366.Central metabolism_rotation3.json',
    reaction_data=solution3.fluxes,
    #reaction_data=solution.fluxes,
)
builder

Builder(reaction_data={'CYTDK2': 0.0, 'XPPT': 0.0, 'HXPRT': 0.0, 'NDPK5': 0.0, 'SHK3Dr': 0.0, 'NDPK6': 0.0, 'N…

In [20]:
mutant.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.0009268,0,0.00%
cl_e,EX_cl_e,0.0009268,0,0.00%
cobalt2_e,EX_cobalt2_e,4.451E-06,0,0.00%
cu2_e,EX_cu2_e,0.0001262,0,0.00%
fe2_e,EX_fe2_e,0.00286,0,0.00%
glc__D_e,EX_glc__D_e,5,6,54.55%
k_e,EX_k_e,0.03476,0,0.00%
mg2_e,EX_mg2_e,0.001545,0,0.00%
mn2_e,EX_mn2_e,0.000123,0,0.00%
mobd_e,EX_mobd_e,1.246E-06,0,0.00%


In [15]:
cobra.io.write_sbml_model(mutant, "result/iML1515_xylitol_strain_tested.xml")